In [1]:
import boto3
import csv

In [2]:
def setup_bucket(s3, bucket_name):
    try:
        s3.create_bucket(Bucket=bucket_name, 
                         CreateBucketConfiguration={'LocationConstraint': 'us-west-2'})
    except Exception as e:
        print(e)
        
    bucket = s3.Bucket(bucket_name)
    bucket.Acl().put(ACL='public-read')
    
# setup_bucket()

In [3]:
def upload_file(s3, bucket_name, file_path, key):
    body = open(file_path, 'rb')
    o = s3.Object(bucket_name, key).put(Body=body, ACL='public-read')

In [4]:
def create_table(dyndb, table_name, key_schema, attr_defs, provisioned_throughput):
    try:
        table = dyndb.create_table(TableName=table_name,
                                  KeySchema=key_schema,
                                  AttributeDefinitions=attr_defs,
                                  ProvisionedThroughput=provisioned_throughput)
    except Exception as e:
        print(e)
        table = dyndb.Table(table_name)
        
    table.meta.client.get_waiter('table_exists').wait(TableName=table_name)
        
    return table

In [5]:
def setup_bucket_and_table(s3, dyndb):
    bucket_name = 'datacont-rvimba-nosql'
    table_name = 'DataTable'
    key_schema = [
        {
            'AttributeName': 'PartitionKey',
            'KeyType': 'HASH'
        }, 
        {
            'AttributeName': 'RowKey',
            'KeyType': 'RANGE'
        }
    ]
    attr_defs = [
        {
            'AttributeName': 'PartitionKey',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'RowKey',
            'AttributeType': 'S'
        }
    ]
    provisioned_throughput = {
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
    
    table = create_table(dyndb, table_name, key_schema, attr_defs, provisioned_throughput)
    
    with open('experiments.csv', 'rt') as csvfile:
        csvf = csv.reader(csvfile)
        header = next(csvf)
        for item in csvf:
            file_path = item[4]
            key = item[4]
        
            upload_file(s3, bucket_name, file_path, key)
            
            url = 'https://datacont-rvimba-nosql.s3.us-west-2.amazonaws.com/'+file_path
            
            metadata_item = {'PartitionKey': 'experiments', 
                             'RowKey': item[0], 
                             'Temp': item[1],
                             'Conductivity': item[2], 
                             'Concentration': item[3], 
                             'url': url
                            }
            try:
                table.put_item(Item=metadata_item)
            except:
                print("item may already be in there or failure")

In [6]:
def query_data(dyndb, table_name):
    table = dyndb.Table(table_name)
    for row in range(1,4):
        response = table.get_item(Key = {'PartitionKey': 'experiments', 'RowKey':str(row)})
        item = response['Item']
        print(item)

In [7]:
def main():
    
    # Note to instructor - please set the following keys:
    ACCESS_KEY = 'XXXXXXXXXXXXXXXXXXXX'
    SECRET_ACCESS_KEY = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
                         
    # create an s3 instance object
    s3 = boto3.resource('s3',
                        aws_access_key_id=ACCESS_KEY, 
                        aws_secret_access_key=SECRET_ACCESS_KEY)

    dyndb = boto3.resource('dynamodb',
                          region_name='us-west-2',
                          aws_access_key_id=ACCESS_KEY,
                          aws_secret_access_key=SECRET_ACCESS_KEY)
    
    setup_bucket_and_table(s3, dyndb)
    
    query_data(dyndb, 'DataTable')
    
main()

An error occurred (ResourceInUseException) when calling the CreateTable operation: Table already exists: DataTable
{'Temp': '-1', 'RowKey': '1', 'Conductivity': '52', 'Concentration': '3.4', 'PartitionKey': 'experiments', 'url': 'https://datacont-rvimba-nosql.s3.us-west-2.amazonaws.com/exp1.csv'}
{'Temp': '-2', 'RowKey': '2', 'Conductivity': '52.1', 'Concentration': '3.4', 'PartitionKey': 'experiments', 'url': 'https://datacont-rvimba-nosql.s3.us-west-2.amazonaws.com/exp2.csv'}
{'Temp': '-2.93', 'RowKey': '3', 'Conductivity': '57.1', 'Concentration': '3.7', 'PartitionKey': 'experiments', 'url': 'https://datacont-rvimba-nosql.s3.us-west-2.amazonaws.com/exp3.csv'}
